In [1]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
# 빈 데이터프레임
kdt_res = pd.DataFrame(index=['기관명', '훈련과정명', '훈련시작일자', '훈련종료일자', '총훈련기간', '총훈련시간', '취업률']).T

In [3]:
# 페이지 반복
for page in range(1,50):
    url = f'''
    https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0300L.do?crseTracseSeSelected=%EC%A0%84%EC%B2%B4&ncsSelected=&a
    reaSelected=%EC%A0%84%EA%B5%AD+%3E+%EC%A0%84%EC%B2%B4&searchTotalKeyword=undefined&crseTracseSe=A%7C%ED%9B
    %88%EB%A0%A8%EC%9C%A0%ED%98%95+%EC%A0%84%EC%B2%B4&area=00%7C%EC%A0%84%EA%B5%AD+%EC%A0%84%EC%B2%B4&pageId=6&
    bgrlInstYn=undefined&currentTab=1&pageIndex={page}&kDgtlYn=undefined&crseTracseSeNum=&pageSize=10&orderKey=2
    &orderBy=ASC&mberId=&mberSe=&pop=&kdgLinkYn=&monthGubun=12&topMenuYn=&gb=&keywordType=1&keyword=&keyword2=&keyw
    ord1=&startDate=20240220&endDate=20250220&i2=A&dghtSe=A&traingMthCd=A&totamtSuptYn=A&tracseId=&tracseTme=&intrst
    InfoAdiCn=&pageOrder=2ASC&pageRow=10&tracseId=AIG20230000455498&tracseTme=3&tracseId=AIG20220000366871&tracseTme=5&
    tracseId=AIG20230000412664&tracseTme=4&tracseId=AIG20230000455731&tracseTme=3&tracseId=AIG20220000366834&tracseTme=2
    &tracseId=AIG20230000455775&tracseTme=2&tracseId=AIG20230000412780&tracseTme=2
    '''
    req = requests.get(url)
    content = req.content
    soup = BeautifulSoup(content, 'html.parser')
    
# 내용 추가
    zone = [re.sub('\n\t+', '', z.text) for z in soup.select('p.zone')]  # 기관명
    name = [re.sub('\n\t+', '', n.text) for n in soup.select('p.tit a')]  # 과정명

    start_day = []  # 시작일자
    end_day = []  # 종료일자

    for d in soup.select('dl.item.term'):
        if re.findall('\d+-\d+-\d+', d.text) == []:
            pass
        else:
            start_day.append(re.findall('\d+-\d+-\d+', d.text)[0])
            end_day.append(re.findall('\d+-\d+-\d+', d.text)[1])

    day = []  # 총훈련일자
    time = []  # 총훈련기간

    for t in soup.select('dl.item.time2'):
        day.append(re.sub(',','', re.findall('\S+', t.text)[1]))
        time.append(re.sub(',','', re.findall('\S+', t.text)[2]))

    rate = [re.sub('\n','',re.sub('\n+\t+', '', a.text)) for a in soup.select('dd.graph')]  # 취업률  

# 페이지별 데이터프레임
    kdt_temp = pd.DataFrame([zone, name, start_day, end_day, day, time, rate],
                            index = ['기관명', '훈련과정명', '훈련시작일자', '훈련종료일자', '총훈련기간', '총훈련시간', '취업률']).T
# 빈 프레임에 추가    
    kdt_res = pd.concat([kdt_res, kdt_temp])

In [12]:
# csv파일 저장
kdt_res.to_csv('./kdt.csv', encoding='cp949', index=False)